<a href="https://colab.research.google.com/github/peeyushsinghal/da/blob/main/mitigating_bias_sa_da_v7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mitigating bias in sentiment analysis using domain adaptation

In [1]:
! pip install torchtext==0.10.0 --quiet # DOWNGRADE YOUR TORCHTEXT
! pip install ekphrasis --quiet # library to pre process twitter data
! pip install emoji --upgrade --quiet #library to deal with emoji data


     |████████████████████████████████| 7.6 MB 15.2 MB/s 
     |████████████████████████████████| 831.4 MB 2.6 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
     |████████████████████████████████| 83 kB 2.4 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 45 kB 3.9 MB/s 
     |████████████████████████████████| 240 kB 13.9 MB/s 


In [2]:
## Import statements
import pandas as pd
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchtext.legacy.data import Dataset, Field, TabularDataset, BucketIterator
from torchtext.vocab import GloVe
# import torchtext.functional as TTF
import numpy as np
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import emoji
from torchtext.legacy.vocab import Vectors
from tqdm import tqdm
import random
import torch.optim as optim
# import json
# Importing library
import scipy.stats as stats
from statistics import mean

In [3]:
# checking device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on:{}".format(DEVICE))

Running on:cuda


## Data loading

In [4]:
#Mounting google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Data Configuration

In [5]:
BASE_PATH = '/content/drive/MyDrive/semeval-2018'

DATA_DIR = os.path.join(BASE_PATH,'datasets')
TARGET_DIR = os.path.join(BASE_PATH,'targetdataset')

MODEL_DIR = os.path.join(BASE_PATH,'models')
REF_DIR = os.path.join(BASE_PATH,'reference')

if not os.path.exists(MODEL_DIR):
  os.makedirs(MODEL_DIR)
  print("The new directory is created!")

domain_source = 0.0
domain_target = 1.0

In [6]:
class TASK1(object):
  
    EI_reg = {
        'anger': {
            'train': os.path.join(
                DATA_DIR, 'task1/EI-reg/training/EI-reg-En-anger-train.txt'),
            'dev': os.path.join(
                DATA_DIR, 'task1/EI-reg/development/2018-EI-reg-En-anger-dev.txt'),
            'gold': os.path.join(
                DATA_DIR, 'task1/EI-reg/test-gold/2018-EI-reg-En-anger-test-gold.txt')
                },
        'joy': {
                'train': os.path.join(
                    DATA_DIR, 'task1/EI-reg/training/EI-reg-En-joy-train.txt'),
                'dev': os.path.join(
                    DATA_DIR, 'task1/EI-reg/development/2018-EI-reg-En-joy-dev.txt'),
                'gold': os.path.join(
                    DATA_DIR, 'task1/EI-reg/test-gold/2018-EI-reg-En-joy-test-gold.txt')
                    }
        }

    # V_reg = {
    #     'train': os.path.join(
    #         DATA_DIR, 'task1/V-reg/2018-Valence-reg-En-train.txt'),
    #     'dev': os.path.join(
    #         DATA_DIR, 'task1/V-reg/2018-Valence-reg-En-dev.txt'),
    #     'gold': os.path.join(
    #         DATA_DIR, 'task1/V-reg/2018-Valence-reg-En-test-gold.txt')
    #          }

    EEC = {
        'eec': os.path.join(
            DATA_DIR, 'task1/Equity-Evaluation-Corpus/Equity-Evaluation-Corpus.csv')
             }

## Source Data
Parsing Emotion and Valence regression data : `format [ID	Tweet	Affect Dimension	Intensity Score]`

In [7]:
def parse_reg(data_file, label_format='tuple'):#->  pd.DataFrame:
    """
    This is for datasets for the EI-reg and V-reg English tasks 
    Returns:
        df: dataframe with columns in the first row of file [ID-Tweet-Affect Dimension-Intensity Score]
    """
    with open(data_file, 'r') as fd:
      data = [l.strip().split('\t') for l in fd.readlines()]
    
    # print(data)
    df = pd.DataFrame (data[1:],columns=data[0])
    csv_file_name = (data_file.split("/")[-1]).split('.')[0]+".csv"
    csv_file = df.to_csv(str(csv_file_name))
    # df['domain'] = domain_source
    return csv_file_name
    # return df

In [8]:
parse_reg(os.path.join(DATA_DIR, 'task1/EI-reg/training/EI-reg-En-joy-train.txt'))

'EI-reg-En-joy-train.csv'

## Source Parser
Generic Source Data Parser

In [9]:
def parse_csv(task, dataset, emotion='anger'):
    if task == 'EI-reg':
        data_train = TASK1.EI_reg[emotion][dataset]
        csv_file_name = parse_reg(data_train)
        return csv_file_name
        # df[df.columns[-1]] = df[df.columns[-1]].astype(float)
        # return df
    # elif task == 'V-reg':
    #     data_train = TASK1.V_reg[dataset]
    #     df = parse_reg(data_train)
    #     df[df.columns[-1]] = df[df.columns[-1]].astype(float)
    #     return df
    else:
        return None

In [10]:
file_EI_reg_train = parse_csv('EI-reg','train','joy')
file_EI_reg_train

file_EI_reg_val = parse_csv('EI-reg','dev','joy')
print(file_EI_reg_val)

file_EI_reg_test = parse_csv('EI-reg','gold','joy')
(file_EI_reg_test)

2018-EI-reg-En-joy-dev.csv


'2018-EI-reg-En-joy-test-gold.csv'

## Preprocess tweets

In [11]:
# reference : https://github.com/cbaziotis/ekphrasis


text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [12]:
# #### Example checks of pre-processing
# sentences = [
#     "CANT WAIT for the new season of #TwinPeaks ＼(^o^)／!!! #davidlynch #tvseries :)))",
#     "I saw the new #johndoe movie and it suuuuucks!!! WAISTED $10... #badmovies :/",
#     "@SentimentSymp:  can't wait for the Nov 9 #Sentiment talks!  YAAAAAAY !!! :-D http://sentimentsymposium.com/.",
#     "@MGBarbieri @SpalkTalk a@b.com And just saw your LinkedIn comment after I sent this! Thanks for the message :) 😀",
#     "💙💛🏆 @GeorgeePitman Young Player of The Season 🏆💛💙 #irony #actuallyseventy"
# ]

# for s in sentences:
#     print(" ".join(text_processor.pre_process_doc(s)))
# # print ([text_processor.pre_process_doc(s) for s in sentences])

In [13]:
def preprocess_tweet(tweet): 
  tweet_processed = text_processor.pre_process_doc(tweet)
  # print (tweet_processed)
  final_list =[]
  for index, tweet in enumerate(tweet_processed):
      final_list.append(emoji.demojize(tweet, language = 'en'))
  
  # print(df)
  return final_list

In [14]:
# #### Example checks of pre-processing
# sentences = [
#     "CANT WAIT for the new season of #TwinPeaks ＼(^o^)／!!! #davidlynch #tvseries :)))",
#     "I saw the new #johndoe movie and it suuuuucks!!! WAISTED $10... #badmovies :/",
#     "@SentimentSymp:  can't wait for the Nov 9 #Sentiment talks!  YAAAAAAY !!! :-D http://sentimentsymposium.com/.",
#     "@MGBarbieri @SpalkTalk a@b.com And just saw your LinkedIn comment after I sent this! Thanks for the message :) 😀",
#     "💙💛🏆 @GeorgeePitman Young Player of The Season 🏆💛💙 #irony #actuallyseventy"
# ]

# for s in sentences:
#   print(preprocess_tweet(s))
#   # print(" ".join(preprocess_tweet(s)))

## TorchText Treatment

In [15]:
MAX_SIZE = 50
field_tweet = Field(sequential=True, use_vocab = True, tokenize = preprocess_tweet, fix_length = MAX_SIZE, batch_first = True)
# field_tweet = Field(sequential=True, use_vocab = False, tokenize = preprocess_tweet)

field_intensity = Field(sequential= False, 
                        dtype = torch.float,
                        use_vocab = False 
                        )

In [16]:
fields = {
    'Tweet':('tweet', field_tweet ), #
    'Intensity Score': ('intensity',field_intensity) # Intensity Score is name of the dataset column, field_intensity is how we have defined the field, intensity is the name of the variable going fwd
    }

In [17]:
train_data, valid_data, test_data = TabularDataset.splits( path = './', 
                                              train = file_EI_reg_train,
                                              validation = file_EI_reg_val, 
                                              test = file_EI_reg_test,
                                              format = 'csv',
                                              fields = fields
                                          )

In [18]:
print(len(train_data))

1616


In [19]:
print(train_data[0].__dict__.keys())

dict_keys(['tweet', 'intensity'])


In [20]:
print(valid_data[0].__dict__.values())

dict_values([['<user>', 'oh', '!', 'that', 'actually', 'was', 'my', 'first', 'guess', 'but', '.', '<repeated>', 'i', 'thought', 'he', 'was', 'too', 'dark', 'for', 'an', 'irish', 'man', 'from', '<allcaps>', 'bce', '</allcaps>', '.', 'thanks', 'for', 'clearing', 'it', 'up', '<laugh>'], '0.470'])


In [21]:
print((train_data[2].__dict__.values()))

dict_values([['positive', '<hashtag>', 'psychology', '</hashtag>', 'research', 'shows', 'salespeople', 'who', 'score', 'in', 'the', 'top', '<percent>', 'for', '<hashtag>', 'optimism', '</hashtag>', 'have', '<percent>', '>', 'sales', 'than', 'those', 'in', 'top', '<percent>', 'for', 'pessimism', '.'], '0.274'])


In [22]:
count = 0
for example in test_data.examples:
  print(example.tweet, example.intensity)
  count += 1
  if count > 10:
    break

['people', 'are', 'truly', '<hashtag>', 'amazing', '</hashtag>', '.', '<hashtag>', 'inspiring', '</hashtag>', 'day'] 0.712
['what', 'are', 'some', 'good', '<hashtag>', 'funny', '</hashtag>', '<hashtag>', 'entertaining', '</hashtag>', '<hashtag>', 'interesting', '</hashtag>', 'accounts', 'i', 'should', 'follow', '?', 'my', 'twitter', 'is', 'dry'] 0.339
['<user>', 'a', 'review', 'of', 'my', 'book', 'faulted', 'me', 'for', 'spending', 'so', 'much', 'time', 'on', 'webster', "'", 's', 'introductory', 'front', 'matter', '.', 'but', 'i', 'had', 'to', ',', '<hashtag>', 'brilliant', '</hashtag>', '!'] 0.533
['<hashtag>', 'good', '</hashtag>', 'to', 'learning', '<hashtag>', 'wisdom', '</hashtag>', '<', '<', 'reform', '(', 'v', ')', ':', 'make', 'in', 'order', 'to', 'improve', 'something', '>', '>'] 0.281
['really', 'excited', 'to', 'see', 'our', 'team', 'this', 'year', ',', 'and', 'especially', 'moving', 'in', 'the', 'next', '<number>', '-', '<number>', 'years', 'when', 'his', 'guys', 'start', '

## Building iterator and Vocabulary

In [23]:
MAX_VOCAB_SIZE = 10000

field_tweet.build_vocab(train_data, 
                  max_size = MAX_VOCAB_SIZE,
                  min_freq = 1,
                  vectors = "glove.6B.100d",
                  unk_init=torch.Tensor.normal_)

vocab = field_tweet.build_vocab(train_data, 
                  max_size = MAX_VOCAB_SIZE,
                  min_freq = 1,
                  vectors = "glove.6B.100d",
                  unk_init=torch.Tensor.normal_)
field_intensity.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [02:40, 5.37MB/s]                           
100%|█████████▉| 399999/400000 [00:13<00:00, 30723.91it/s]


In [24]:
type(vocab)

NoneType

In [25]:
print(len(valid_data),len(test_data))


290 1105


In [34]:
VALID_TEST_BATCH_SIZE = min (len(valid_data),len(test_data))
# print(VALID_TEST_BATCH_SIZE)
BATCH_SIZE = 8
train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_data, valid_data, test_data), 
                                                      batch_sizes= (BATCH_SIZE,VALID_TEST_BATCH_SIZE,VALID_TEST_BATCH_SIZE),
                                                      sort_key = lambda x: len(x.tweet),
                                                      sort_within_batch=True,
                                                      device = DEVICE,
                                                      shuffle= True)

In [37]:
# next(iter(valid_iterator))


[torchtext.legacy.data.batch.Batch of size 290]
	[.tweet]:[torch.cuda.LongTensor of size 290x50 (GPU 0)]
	[.intensity]:[torch.cuda.FloatTensor of size 290 (GPU 0)]

In [ ]:
count = 0
for batch in train_iterator:
  print (batch.tweet)
  print (batch.intensity)
  count += 1
  if count > 2:
    break

In [ ]:
print(list(field_tweet.vocab.stoi.items()))

In [ ]:
print(field_tweet.vocab.vectors[field_tweet.vocab.stoi['the']])

In [ ]:
field_tweet.vocab.vectors[9]

In [ ]:
# for batch_number, batch in enumerate(train_iterator):
#   print(batch_number)
#   print(batch.tweet)
#   print(batch.intensity)

## TorchText treatment of Target Data

In [39]:
# Target Data
df_target = pd.read_csv(os.path.join(TARGET_DIR,'winomt_saunders_combined.csv'))
# df_target.columns = ['Sentence']
# df_target.columns = ['Tweet']
# df_target['Intensity Score'] = 0
print(df_target.head())

target_file_name = 'target_csv.csv'
target_csv_file = df_target.to_csv(str(target_file_name))

                             Tweet  Intensity Score
0    the actor finished her work .                0
1    the actor finished his work .                0
2  the actuary finished her work .                0
3  the actuary finished his work .                0
4    the agent finished her work .                0


In [40]:
target_data = TabularDataset( path = os.path.join(TARGET_DIR,'winomt_saunders_combined.csv'), 
                                              format = 'csv',
                                              fields = fields
                                          )
print(target_data[0].__dict__.keys())
print(target_data[0].__dict__.values())

dict_keys(['tweet', 'intensity'])
dict_values([['the', 'actor', 'finished', 'her', 'work', '.'], '0'])


In [41]:
# target_data, _ = TabularDataset.splits( path = './', 
#                                               train = target_file_name,
#                                               test = target_file_name, # arbitary file
#                                               format = 'csv',
#                                               fields = fields
#                                           )
# print(target_data[0].__dict__.keys())
# print(target_data[0].__dict__.values())

In [42]:
count = 0
for example in target_data:
  print(example.tweet, example.intensity)
  count += 1
  if count > 2:
    break

['the', 'actor', 'finished', 'her', 'work', '.'] 0
['the', 'actor', 'finished', 'his', 'work', '.'] 0
['the', 'actuary', 'finished', 'her', 'work', '.'] 0


In [ ]:

# TARGET_BATCH_SIZE = 2
# target_iterator, _ = BucketIterator.splits((target_data, target_data), 
#                                                       batch_size= TARGET_BATCH_SIZE,
#                                                       sort_key = lambda x: len(x.tweet),
#                                                       sort_within_batch=True,
#                                                       device = DEVICE,
#                                                       shuffle= True)

In [43]:
TARGET_BATCH_SIZE = 2
target_iterator = BucketIterator(target_data, # given that there is only one dataset we are not using splits
                                 batch_size= TARGET_BATCH_SIZE,
                                 sort_key = lambda x: len(x.tweet),
                                 sort_within_batch=True,
                                 device = DEVICE,
                                 repeat=True,
                                 shuffle= True)

In [ ]:
len(target_data), len(target_iterator)

In [ ]:
len(target_data)//TARGET_BATCH_SIZE

In [ ]:
next(iter(target_iterator)).tweet

In [44]:
count = 0
for batch in target_iterator:
  print(batch)
  print (batch.tweet)
  print (batch.intensity)
  count += 1
  if count > 2:
    break


[torchtext.legacy.data.batch.Batch of size 2]
	[.tweet]:[torch.cuda.LongTensor of size 2x50 (GPU 0)]
	[.intensity]:[torch.cuda.FloatTensor of size 2 (GPU 0)]
tensor([[   6,    0,    0,  190,    9,  552, 1249,    0,    0,   10,   46,  104,
           21,   54,    6,    0,   70,  352,  194,    4,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1],
        [   6,    0,  940,    9,  699,    6,    0,  104,   59,  143,  108, 1194,
           71,   89, 4134,   20,    6,    0,    0,    4,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1]], device='cuda:0')
tensor([0., 0.], device='cuda:0')

[torchtext.legacy.data.batch.Batch of size 2]
	[.tweet]:[torch.cuda.LongTensor of size 2x50 (GPU

## CNN 1d model

### Gradient Reversal layer

In [45]:
from torch.autograd import Function


class GradientReversalFn(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

CNN 1 D model
Reference: A Sensitivity Analysis of (and Practitioners' Guide to) Convolutional Neural Networks for Sentence Classification, Ye Zhang, Byron Wallace 2015

Difference:

use of embedding
use of sigmoid function, as we are having a regression model not a classififer as the main task

In [46]:
import torch.nn as nn
import torch.nn.functional as F

class CNN1d(nn.Module):
    def __init__(self, 
                 vocab_size, 
                 embedding_dim, 
                 n_filters, 
                 filter_sizes, 
                 output_dim, 
                 dropout, 
                 pad_idx
                 ):
        super().__init__()
        
        #---------------------Feature Extractor Network----------------------#
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)

        # Convolutional Network
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = embedding_dim, 
                                              out_channels = n_filters, 
                                              kernel_size = fs)
                                    for fs in filter_sizes
                                    ])
        
        #---------------------Regression Network------------------------#
        # Fully-connected layer and Dropout
        self.regression = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(len(filter_sizes) * n_filters, len(filter_sizes) * n_filters // 2),
            nn.ReLU(),
            nn.Linear(len(filter_sizes) * n_filters // 2, output_dim * 10),
            nn.ReLU(),
            nn.Linear(output_dim * 10, output_dim)
            # ,
            # nn.Sigmoid()
        )
        # self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim * 10)
        # self.fc2 = nn.Linear(output_dim * 10, output_dim)
        # self.dropout = nn.Dropout(dropout)

        #---------------------Domain Classifier Network------------------------#
        # Fully-connected layer and Dropout
        self.domain_classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(len(filter_sizes) * n_filters, len(filter_sizes) * n_filters // 2),
            nn.ReLU(),
            nn.Linear(len(filter_sizes) * n_filters // 2, output_dim * 10),
            nn.ReLU(),
            nn.Linear(output_dim * 10, 2),
            nn.LogSoftmax(dim=1),
        )
        
    def forward(self, text, alpha=1.0):
        
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.permute(0, 2, 1)
        
        #embedded = [batch size, emb dim, sent len]
        
        conved = [F.relu(conv(embedded)) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        x_feature = torch.cat(pooled, dim = 1)
        
        #x_feature = [batch size, n_filters * len(filter_sizes)]
        
        reverse_feature = GradientReversalFn.apply(x_feature, alpha)
        # print("reverse_feature",reverse_feature)
    
        regression_output = self.regression(x_feature)
    
        domain_classifier_output = self.domain_classifier(reverse_feature)


        return regression_output, domain_classifier_output

In [47]:
INPUT_DIM = len(field_tweet.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2, 3, 4, 5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = field_tweet.vocab.stoi[field_tweet.pad_token]

In [48]:
model = CNN1d(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
model.to(DEVICE)

CNN1d(
  (embedding): Embedding(4788, 100, padding_idx=1)
  (convs): ModuleList(
    (0): Conv1d(100, 100, kernel_size=(2,), stride=(1,))
    (1): Conv1d(100, 100, kernel_size=(3,), stride=(1,))
    (2): Conv1d(100, 100, kernel_size=(4,), stride=(1,))
    (3): Conv1d(100, 100, kernel_size=(5,), stride=(1,))
  )
  (regression): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=400, out_features=200, bias=True)
    (2): ReLU()
    (3): Linear(in_features=200, out_features=10, bias=True)
    (4): ReLU()
    (5): Linear(in_features=10, out_features=1, bias=True)
  )
  (domain_classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=400, out_features=200, bias=True)
    (2): ReLU()
    (3): Linear(in_features=200, out_features=10, bias=True)
    (4): ReLU()
    (5): Linear(in_features=10, out_features=2, bias=True)
    (6): LogSoftmax(dim=1)
  )
)

### Load Pre trained embeddings
we'll load the pre-trained *embeddings*

In [49]:
pretrained_embeddings = field_tweet.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-1.6710, -0.2917, -0.7022,  ..., -1.0392, -0.3673, -0.7857],
        [ 1.4578, -0.6760,  0.5083,  ...,  1.0674, -1.2953, -0.2963],
        [ 2.9691,  0.0103,  0.2255,  ...,  1.4566,  0.6065,  0.3366],
        ...,
        [-2.4249,  0.2149, -1.0821,  ..., -1.0615,  1.8110, -0.8679],
        [ 0.7447,  1.7536, -0.1841,  ...,  0.7329,  0.0480,  0.7125],
        [-1.0397,  0.3304, -0.6786,  ..., -1.1080, -0.5790,  1.2534]],
       device='cuda:0')

In [50]:
field_tweet.vocab.vectors.shape

torch.Size([4788, 100])

In [51]:
UNK_IDX = field_tweet.vocab.stoi[field_tweet.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

## Training the model

### Without training one forward pass

In [52]:
for batch in train_iterator:
  print(batch.tweet)
  output = model(batch.tweet)
  print (output)
  break

tensor([[   8,  141,   61,    9, 2722,    6,  113,   16, 4194, 2386,   10,   31,
           12,   10,  156, 1144,   46,    4,  233,   13,   56,   36,   10,  125,
           80,   14,    3, 4767, 4766,    2,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1],
        [   5,    8,  298,  258,  231,    7,   41,  369, 1117, 4063,   20,  622,
           72,    7,    8,   11,  254,  223,    7, 3631,   34, 1282,  957,   19,
            9,  666,  959,  957,   19,    4,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1],
        [   6,  294,  602,  208,   20,    6,  871,   16,  921,   15,  573,  161,
           59,   45,  900,    7,   59,   45,  124,  112,  921,   32,  245,  161,
           24, 1873,  188,   43, 3453,   42,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,   

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [ ]:
# import torch.optim as optim

# optimizer = optim.Adam(model.parameters())

# criterion = nn.BCEWithLogitsLoss()

# model = model.to(DEVICE)
# criterion = criterion.to(DEVICE)

### Typical Train Model Function

In [53]:
# Typical Training Function

from tqdm import tqdm # for beautiful model training updates


def train_model(model, device, train_loader, optimizer, epoch):
    model.train() # setting the model in training mode
    pbar = tqdm(train_loader) # putting the iterator in pbara
    correct = 0 # for accuracy numerator
    processed =0 # for accuracy denominator
    epoch_loss = 0.0
    for batch_idx, batch in enumerate(pbar):

        tweets, intensities = batch.tweet.to(device), batch.intensity.to(device)  # plural, we are not interested in domain
        #sending data to CPU or GPU as per device

        optimizer.zero_grad() # setting gradients to zero to avoid accumulation

        y_preds,_ = model(tweets) # forward pass, result captured in y_preds (plural as there are many body in a batch)
        # we are not interested in domain prediction
        # the predictions are in one hot vector

        regression_loss = regression_loss_function(y_preds,intensities.unsqueeze(1)) # Computing loss
        # loss = F.mse_loss(y_preds,intensities.unsqueeze(1)) # Computing loss

        train_regresion_losses.append(regression_loss.item()) # to capture loss over many epochs

        regression_loss.backward() # backpropagation
        optimizer.step() # updating the params

        # preds = y_preds.argmax(dim=1, keepdim=True)  # get the index olf the max log-probability
        # correct += preds.eq(labels.view_as(preds)).sum().item()
        epoch_loss += regression_loss.item()

        processed += len(tweets)

        pbar.set_description(desc= f'Loss={regression_loss.item()} Batch_id={batch_idx} Epoch Average loss={100*epoch_loss/processed:0.4f}')
    train_accuracy.append(100*epoch_loss/len(train_loader))

### Typical Test Function

In [54]:
def test_model(model,device, data_loader, mode= 'test'):
    model.eval() # setting the model in evaluation mode
    loss = 0
    correct = 0 # for accuracy numerator
    test_regresion_losses =[] # for overall batches (summed over batches)
    valid_regresion_losses =[] # for overall batches (summed over batches)

    with torch.no_grad():
        for batch in data_loader:

            tweets, intensities  = batch.tweet.to(device), batch.intensity.to(device) #sending data to CPU or GPU as per device
            # we are not interested in domains
            
            y_preds,_ = model(tweets) # forward pass, result captured in outputs (plural as there are many bodies in a batch)
            # the outputs are in batch size x one hot vector 
            # not interested in domain output

            regression_loss = regression_loss_function(y_preds,intensities.unsqueeze(1))

            if mode == 'test':
              test_regresion_losses.append(regression_loss.item())
              # print(f'...in the batch...{regression_loss}')
            else:
              valid_regresion_losses.append(regression_loss.item())
              # print(f'...in the batch...{regression_loss}')

        # regression_loss.item() /= len(data_loader.dataset) # average test loss
        if mode == 'test':
          # total_test_regression_loss = sum(test_regresion_losses)
          # test_regresion_losses.append(regression_loss) # to capture loss over many batches
          # print('...Average test loss: {:.8f}'.format((total_test_regression_loss)/len(data_loader.dataset)))
          print(f'TEST LOSS (Average) : {sum(test_regresion_losses) / len(data_loader)}')
        else:
          # valid_regresion_losses.append(regression_loss) # to capture loss over many batches
          # total_valid_regression_loss = sum(valid_regresion_losses)
          # print('...Average validation loss: {:.8f}'.format((total_valid_regression_loss)/len(data_loader.dataset)))
          print(f'VALIDATION LOSS (Average) : {sum(valid_regresion_losses) / len(data_loader)}')

In [55]:
# EXECUTION

lr = 2e-5
optimizer = optim.Adam(model.parameters(), lr=lr)
domain_loss_function= nn.BCEWithLogitsLoss()
regression_loss_function = nn.L1Loss()


model = model.to(DEVICE)
domain_loss_function = domain_loss_function.to(DEVICE)
regression_loss_function = regression_loss_function.to(DEVICE)

# train_losses = [] # to capture train losses over training epochs
train_accuracy = [] # to capture train accuracy over training epochs
# val_losses = [] # to capture validation loss
# test_losses = [] # to capture test losses 
# test_accuracy = [] # to capture test accuracy 

# EPOCHS = 2
EPOCHS = 100
# dict_val_loss = {}
# dict_test_loss = {}


train_regresion_losses = [] # to capture train losses over training epochs
train_domain_losses = []
train_accuracy = [] # to capture train accuracy over training epochs
# valid_regresion_losses = [] # to capture validation loss
# test_regresion_losses = [] # to capture test losses 
total_test_regression_loss =[]
total_valid_regression_loss =[]
# print(f'----------------------training started for {name}-----------------')
for epoch in range(EPOCHS):
  print("EPOCH:", epoch+1)
  train_model(model, DEVICE, train_iterator, optimizer, epoch)
  # print("for validation.......")
  # val_name = train_name.replace("train", "val" )
  # test_model(typical_model, device, dict_val_loader[val_name], mode = 'val')
  test_model(model, DEVICE, valid_iterator, mode = 'val')


  # print("for test  .......")
  # test_name = train_name.replace("train", "test" )
  # test_model(typical_model, device, dict_test_loader[test_name], mode = 'test')
  test_model(model, DEVICE, test_iterator, mode = 'test')

# dict_val_loss[name] = val_losses
# dict_test_loss[name] = test_losses

model_name = "Non_DANN"+".pt"
torch.save(model.state_dict(), os.path.join(MODEL_DIR, model_name))
# print(f'----------------------training complete for {name}-----------------')
# print(dict_val_loss.items())
# print(dict_test_loss.items())

EPOCH: 1


Loss=0.10554364323616028 Batch_id=201 Epoch Average loss=2.1569: 100%|██████████| 202/202 [00:01<00:00, 120.95it/s]


for validation.......
...Average batch validation loss is 0.1546570360660553
for test  .......
...Average batch test loss is 0.14828822761774063
EPOCH: 2


Loss=0.13246333599090576 Batch_id=201 Epoch Average loss=1.8359: 100%|██████████| 202/202 [00:01<00:00, 128.24it/s]


for validation.......
...Average batch validation loss is 0.13858754932880402
for test  .......
...Average batch test loss is 0.13643446005880833
EPOCH: 3


...

for validation.......
...Average batch validation loss is 0.13651412725448608
for test  .......
...Average batch test loss is 0.12829741649329662
EPOCH: 99


Loss=0.033625997602939606 Batch_id=201 Epoch Average loss=0.3309: 100%|██████████| 202/202 [00:02<00:00, 100.74it/s]


for validation.......
...Average batch validation loss is 0.13567854464054108
for test  .......
...Average batch test loss is 0.12754332087934017
EPOCH: 100


Loss=0.03215210884809494 Batch_id=201 Epoch Average loss=0.3229: 100%|██████████| 202/202 [00:01<00:00, 102.61it/s]


for validation.......
...Average batch validation loss is 0.13678020238876343
for test  .......
...Average batch test loss is 0.128689618781209


## DANN Model - Training and Testing

### Training

In [50]:
# b = next(iter(train_iterator))
# print(b.tweet, b.intensity)

tensor([[   5,  488,    7,  122,    7,  199, 4073, 1832,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1],
        [ 111,  306, 2665,    9,  811,  128,  264,  529,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1]]) tensor([0.4400, 0.2080])


In [57]:
def compute_accuracy(logits, labels):
    
    predicted_labels_dict = {
      0: 0,
      1: 0,
    }
    
    predicted_label = logits.max(dim = 1)[1]
    
    for pred in predicted_label:
        predicted_labels_dict[pred.item()] += 1
    acc = (predicted_label == labels).float().mean()
    
    return acc, predicted_labels_dict

In [107]:
# def binary_acc(y_pred, y_test):
#     y_pred_tag = torch.round(torch.sigmoid(y_pred))

#     correct_results_sum = (y_pred_tag == y_test).sum().float()
#     acc = correct_results_sum/y_test.shape[0]
#     acc = torch.round(acc * 100)
    
#     return acc

In [ ]:
# def evaluate(model, dataloader, mode = 'test', percentage = 5):
#     with torch.no_grad():
#         predicted_labels_dict = {                                                   
#           0: 0,                                                                     
#           1: 0,                                                                     
#         }
        
#         mean_accuracy = 0.0
#         # total_batches = len(dataloader)
#         # print("total_batches: ",total_batches )

        



#             sentiment_pred, _ = model(**inputs)
#             accuracy, predicted_labels = compute_accuracy(sentiment_pred, inputs["labels"])
#             mean_accuracy += accuracy
#             predicted_labels_dict[0] += predicted_labels[0]
#             predicted_labels_dict[1] += predicted_labels[1]  
#         print(predicted_labels_dict)
#     return mean_accuracy/total_batches

In [58]:
 ## DANN training function attempt 2 with NLLLoss
n_epochs = 100 # number of epochs
lr = 2e-5
optimizer = optim.Adam(model.parameters(), lr=lr)

# loss_fn_sentiment_regression = torch.nn.NLLLoss()
# loss_fn_domain_classifier = torch.nn.NLLLoss()

model = model.to(DEVICE)
domain_loss_function= nn.NLLLoss()
regression_loss_function = nn.L1Loss()
domain_loss_function = domain_loss_function.to(DEVICE)
regression_loss_function = regression_loss_function.to(DEVICE)

max_batches = min(len(train_iterator), len(target_data)//TARGET_BATCH_SIZE)
# max_batches = min(len(train_iterator), len(target_iterator))

print(max_batches)

for epoch_idx in range(n_epochs):
    
    source_iterator = iter(train_iterator)
    target_iterator = iter(target_iterator)

    for batch_idx in range(max_batches):
        
        p = float(batch_idx + epoch_idx * max_batches) / (n_epochs * max_batches)
        alpha = 2. / (1. + np.exp(-10 * p)) - 1
        alpha = torch.tensor(alpha)
        
        model.train()
        
        # if(batch_idx%100 == 0 ):
        #     print("Training Step:", batch_idx)
        
        optimizer.zero_grad()

        ## SOURCE DATASET TRAINING UPDATE
        
        source_batch = next(source_iterator)
        source_tweets, source_intensities = source_batch.tweet.to(DEVICE), source_batch.intensity.to(DEVICE)  # plural, we are not interested in domain
        
        source_intensity_outputs, source_domain_outputs = model(source_tweets, alpha = alpha)

        loss_source_regression= regression_loss_function(source_intensity_outputs,source_intensities.unsqueeze(1)) # Computing regression loss

        source_domain_inputs = torch.zeros(len(source_batch), dtype=torch.long).to(DEVICE) # source domain has 0 id
        loss_source_domain = domain_loss_function(source_domain_outputs,source_domain_inputs)


        ## TARGET DATASET TRAINING UPDATE
        target_batch = next(iter(target_iterator))
        target_tweets= target_batch.tweet.to(DEVICE) # plural

        _, target_domain_outputs = model(target_tweets, alpha = alpha)

        target_domain_inputs = torch.ones(len(target_batch), dtype=torch.long).to(DEVICE) # target domain has 1 id
        loss_target_domain = domain_loss_function(target_domain_outputs,target_domain_inputs)

        # COMBINING LOSS
        loss = loss_source_regression + loss_source_domain + loss_target_domain
        loss.backward()
        optimizer.step()

        if (batch_idx % 100 == 0):
          print("Epoch [{}/{}] Step [{}/{}]: domain_loss_target={:.4f} / domain_loss_source={:.4f} / regression_loss_source={:.4f} / alpha={:.4f}"
              .format(epoch_idx + 1,
                      n_epochs,
                      batch_idx + 1,
                      max_batches,
                      loss_target_domain.item()
                      ,loss_source_domain.item()
                      ,loss_source_regression.item(),alpha))


    # Evaluate the model after every epoch


    print("for validation.......")
    test_model(model, DEVICE, valid_iterator, mode = 'val')
    print("for test  .......")
    test_model(model, DEVICE, test_iterator, mode = 'test')

  
torch.save(model.state_dict(), os.path.join(MODEL_DIR, "epoch_" + str(epoch_idx)  +  ".pt" ))


202
Epoch [1/100] Step [1/202]: domain_loss_target=0.9360 / domain_loss_source=0.4043 / regression_loss_source=0.0326 / alpha=0.0000
for validation.......
...Average batch validation loss is 0.1373431384563446
for test  .......
...Average batch test loss is 0.12931600026786327
Epoch [2/100] Step [1/202]: domain_loss_target=1.0782 / domain_loss_source=0.3466 / regression_loss_source=0.0205 / alpha=0.0500
for validation.......
...Average batch validation loss is 0.13746392726898193
for test  .......
...Average batch test loss is 0.12958327867090702
Epoch [3/100] Step [1/202]: domain_loss_target=0.7410 / domain_loss_source=0.2396 / regression_loss_source=0.0179 / alpha=0.0997
for validation.......
...Average batch validation loss is 0.13802608847618103
for test  .......
...Average batch test loss is 0.13033027946949005
Epoch [4/100] Step [1/202]: domain_loss_target=0.7228 / domain_loss_source=0.3157 / regression_loss_source=0.0229 / alpha=0.1489
for validation.......
...Average batch vali

In [111]:
# ## DANN training function attempt 1 with BCE

# from tqdm import tqdm # for beautiful model training updates


# def train_dann_model(model, 
#                      device, 
#                      train_loader, 
#                      target_loader, # this is the target data source
#                      optimizer, 
#                      epoch, 
#                      num_epochs # this is done for alpha parameter tuning
#                      ):
#     model.train() # setting the model in training mode
#     len_dataloader = min(len(train_loader), len(target_loader)) # training for minimum of two dataloaders
    
#     m = nn.LogSoftmax(dim=1) ## to be used with NLLLoss

#     epoch_loss=0
#     epoch_acc=0
#     i = 0 # as the training progresses the alpha changes
#     while i < len_dataloader -1:
#         # implementation of alpha as per paper
#         p = float(i + epoch * len_dataloader) / (num_epochs * len_dataloader)
#         alpha = 2. / (1. + np.exp(-10 * p)) - 1
#         alpha = torch.tensor(alpha)

#         optimizer.zero_grad() # setting gradients to zero to avoid accumulation

#         # training model using source data
#         source_batch = next(iter(train_loader))
#         source_tweets, source_intensities = source_batch.tweet.to(device), source_batch.intensity.to(device)  # plural, we are not interested in domain
#         #sending data to CPU or GPU as per device

#         y_preds, source_domain_outputs = model(source_tweets, alpha = alpha) # forward pass, regresion result captured in y_preds, classificaiton in source_domain_outputs (plural as there are many tweets in a source batch)
#         # we are interested in domain prediction
        
#         loss_source_regression= regression_loss_function(y_preds,source_intensities.unsqueeze(1)) # Computing loss
#         # loss = F.mse_loss(y_preds,intensities.unsqueeze(1)) # Computing loss

#         source_domain_inputs = torch.zeros(len(source_batch), dtype=torch.float).to(device)

#         # print(f'source_domain_outputs: {source_domain_outputs} with {source_domain_outputs.shape}  , source_domain_inputs: {source_domain_inputs.unsqueeze(1)} with {source_domain_inputs.unsqueeze(1).shape}')
#         loss_source_domain = domain_loss_function(source_domain_outputs,source_domain_inputs.unsqueeze(1)) # classification loss
#         # loss_source_domain = domain_loss_function(m(source_domain_outputs).argmax(),source_domain_inputs.unsqueeze(1)) # classification loss to work with NLLLoss, we need log softmax

#         acc = binary_acc(source_domain_outputs, source_domain_inputs.unsqueeze(1))
#         epoch_acc += acc.item()
      
#         ###--------------------------###
#         # training model using target data
#         target_batch = next(iter(target_loader))
#         target_tweets = target_batch.tweet.to(device)  # plural, we are not interested in tweet only

#         _, target_domain_outputs = model(target_tweets, alpha = alpha) # forward pass, regresion result captured in y_preds, classificaiton in source_domain_outputs (plural as there are many tweets in a source batch)

#         target_domain_inputs = torch.ones(len(target_batch), dtype=torch.float).to(device)
#         # print(f'target_domain_inputs: {target_domain_inputs}')
#         loss_target_domain = domain_loss_function(target_domain_outputs, target_domain_inputs.unsqueeze(1)) # classification loss
#         # loss_target_domain = domain_loss_function(m(target_domain_outputs).argmax(), target_domain_inputs.unsqueeze(1)) # classification loss to work with NLLLoss, we need log softmax

#         # print(f' loss_source_regression: {loss_source_regression}, loss_source_domain: {loss_source_domain} , loss_target_domain: {loss_target_domain}')
#         total_loss = loss_source_regression + loss_source_domain + loss_target_domain

#         total_loss.backward() # backpropagation
#         optimizer.step() # updating the params

#         if ((i + 1) % 50 == 0):
#                 print("Epoch [{}/{}] Step [{}/{}]: domain_loss_target={:.4f} / domain_loss_source={:.4f} / regression_loss_source={:.4f}"
#                       .format(epoch + 1,
#                               num_epochs,
#                               i + 1,
#                               len_dataloader,
#                               loss_target_domain.item()
#                               ,loss_source_domain.item()
#                               ,loss_source_regression.item()))
        
#         i = i + 1
#     print (epoch_loss, epoch_acc)

### Evaluation

In [112]:
# # EXECUTION DAANN


# optimizer = optim.Adam(model.parameters(), lr=0.001)
# domain_loss_function= nn.BCEWithLogitsLoss()
# # domain_loss_function= nn.NLLLoss()
# regression_loss_function = nn.L1Loss()


# model = model.to(DEVICE)
# domain_loss_function = domain_loss_function.to(DEVICE)
# regression_loss_function = regression_loss_function.to(DEVICE)

# # train_losses = [] # to capture train losses over training epochs
# train_accuracy = [] # to capture train accuracy over training epochs
# # val_losses = [] # to capture validation loss
# # test_losses = [] # to capture test losses 
# # test_accuracy = [] # to capture test accuracy 

# EPOCHS = 2
# # EPOCHS = 100


# # train_regresion_losses = [] # to capture train losses over training epochs
# # train_domain_losses = []
# # train_accuracy = [] # to capture train accuracy over training epochs
# # # valid_regresion_losses = [] # to capture validation loss
# # # test_regresion_losses = [] # to capture test losses 
# # total_test_regression_loss =[]
# # total_valid_regression_loss =[]
# # print(f'----------------------training started for {name}-----------------')
# for epoch in range(EPOCHS):
#   print("EPOCH:", epoch+1)
#   train_dann_model(model, DEVICE, train_iterator, target_iterator, optimizer, epoch, num_epochs = EPOCHS)

#   print("for validation.......")
#   # val_name = train_name.replace("train", "val" )
#   # test_model(typical_model, device, dict_val_loader[val_name], mode = 'val')
#   # test_model(model, DEVICE, valid_iterator, mode = 'val')


#   print("for test  .......")
#   # test_name = train_name.replace("train", "test" )
#   # test_model(typical_model, device, dict_test_loader[test_name], mode = 'test')
#   # test_model(model, DEVICE, test_iterator, mode = 'test')

# # dict_val_loss[name] = val_losses
# # dict_test_loss[name] = test_losses

# model_name = "DANN"+".pt"
# torch.save(model.state_dict(), os.path.join(MODEL_DIR, model_name))

EPOCH: 1
Epoch [1/2] Step [50/808]: domain_loss_target=0.6931 / domain_loss_source=0.6931 / regression_loss_source=0.0378
Epoch [1/2] Step [100/808]: domain_loss_target=0.6931 / domain_loss_source=0.6931 / regression_loss_source=0.0252
Epoch [1/2] Step [150/808]: domain_loss_target=0.6931 / domain_loss_source=0.6931 / regression_loss_source=0.0383
Epoch [1/2] Step [200/808]: domain_loss_target=0.6931 / domain_loss_source=0.6931 / regression_loss_source=0.0514
Epoch [1/2] Step [250/808]: domain_loss_target=0.6931 / domain_loss_source=0.6931 / regression_loss_source=0.0705
Epoch [1/2] Step [300/808]: domain_loss_target=0.6931 / domain_loss_source=0.6931 / regression_loss_source=0.0618
Epoch [1/2] Step [350/808]: domain_loss_target=0.6931 / domain_loss_source=0.6931 / regression_loss_source=0.0480
Epoch [1/2] Step [400/808]: domain_loss_target=0.6931 / domain_loss_source=0.6931 / regression_loss_source=0.0519
Epoch [1/2] Step [450/808]: domain_loss_target=0.6931 / domain_loss_source=0.693

# Equality Evaluation using Equity Evaluation Corpus

## Loading Evaluation Data
format `[ID	Sentence	Template	Person	Gender	Race Emotion	Emotion word]`

In [60]:
data_EEC = TASK1.EEC['eec']
df_EEC = pd.read_csv(data_EEC)

In [61]:
df_EEC.head()

,ID,Sentence,Template,Person,Gender,Race,Emotion,Emotion word
0,2018-En-mystery-05498,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,angry
1,2018-En-mystery-11722,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,furious
2,2018-En-mystery-11364,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,irritated
3,2018-En-mystery-14320,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,enraged
4,2018-En-mystery-14114,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,annoyed


## Creating evalution function (includes pre-processing)

In [62]:
## padding function : adds padding / truncates to max size
def pad_or_truncate(some_list, target_len = MAX_SIZE ,pad_idx = PAD_IDX):
    return some_list[:target_len] + [pad_idx]*(target_len - len(some_list))

## preprocessing function, takes in a tweet and returns padded indexed tweet (input for model)
def text_pipeline(tweet):
    indexed_tweet = [field_tweet.vocab.__getitem__(token) for token in preprocess_tweet(tweet)]
    # print(indexed_tweet)
    return pad_or_truncate(indexed_tweet, MAX_SIZE , pad_idx = PAD_IDX)
    # print(indexed_tweet_padded)

In [63]:
# i = random.randint(0,len(df_EEC))
# tweet_example = df_EEC['Sentence'][i]
# print(tweet_example, text_pipeline(tweet_example))

## Loading model

In [88]:
### Loading Model
# model_name = "Non_DANN.pt"
# model_name ="epoch_99.pt"
# loaded_model = CNN1d(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
# # loaded_model.to(DEVICE)
# loaded_model.load_state_dict(torch.load(os.path.join(MODEL_DIR, model_name)))
# loaded_model.eval()


dict_model_name = {'non_dann':'Non_DANN.pt','dann':'epoch_99.pt'}
dict_loaded_model ={}
for model_type, model_name in dict_model_name.items():
  loaded_model = CNN1d(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
  loaded_model.load_state_dict(torch.load(os.path.join(MODEL_DIR, model_name)))
  loaded_model.eval()
  dict_loaded_model[model_type] = loaded_model
print(dict_loaded_model)



{'non_dann': CNN1d(
  (embedding): Embedding(4788, 100, padding_idx=1)
  (convs): ModuleList(
    (0): Conv1d(100, 100, kernel_size=(2,), stride=(1,))
    (1): Conv1d(100, 100, kernel_size=(3,), stride=(1,))
    (2): Conv1d(100, 100, kernel_size=(4,), stride=(1,))
    (3): Conv1d(100, 100, kernel_size=(5,), stride=(1,))
  )
  (regression): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=400, out_features=200, bias=True)
    (2): ReLU()
    (3): Linear(in_features=200, out_features=10, bias=True)
    (4): ReLU()
    (5): Linear(in_features=10, out_features=1, bias=True)
  )
  (domain_classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=400, out_features=200, bias=True)
    (2): ReLU()
    (3): Linear(in_features=200, out_features=10, bias=True)
    (4): ReLU()
    (5): Linear(in_features=10, out_features=2, bias=True)
    (6): LogSoftmax(dim=1)
  )
), 'dann': CNN1d(
  (embedding): Embedding(4788, 100, padding_idx=1)
 

In [89]:
from torch.cuda import Device
def predict(tweet, model, text_pipeline,device = DEVICE):
  with torch.no_grad():
    tweet_tensor = torch.tensor(text_pipeline(tweet)).unsqueeze(0).to(device)
    # output = None
    # print(tweet_tensor.shape)
    output = model(tweet_tensor)
    return output[0].item()

In [90]:
i = random.randint(0,len(df_EEC))
tweet_example = df_EEC['Sentence'][i]
loaded_model_device = 'cpu'
model = loaded_model.to(loaded_model_device)
print(predict(tweet_example, loaded_model,text_pipeline, device= loaded_model_device))

0.10847687721252441


## Creating Sentence pairs (as per SEMVAL18 paper)

In [67]:
dict_f_m_noun_phrase = {'she':'he', 
            'her':'him',
            'this woman':'this man',
            'this girl':'this boy',
            'my sister' : 'my brother',
            'my daughter' : 'my son',
            'my wife': 'my husband',
            'my girlfriend':'my boyfriend',
            'my mother':'my father',
            'my aunt':'my uncle',
            'my mom': 'my dad'
            }

name_male = ['Alonzo','Jamel','Alphonse','Jerome','Leroy','Torrance','Darnell','Lamar','Malik','Terrence','Adam','Harry','Josh','Roger','Alan','Frank','Justin','Ryan','Andrew','Jack'] 
name_female = ['Nichelle','Shereen','Ebony','Latisha','Shaniqua','Jasmine','Tanisha','Tia','Lakisha','Latoya','Amanda','Courtney','Heather','Melanie','Katie','Betsy','Kristin','Nancy','Stephanie','Ellen']


In [68]:
list_unique_template = list(df_EEC['Template'].dropna().unique())
# print(list_unique_template)
list_emotion_word = list(df_EEC['Emotion word'].unique())
# print(list_emotion_word)
list_gender = list(df_EEC['Gender'].dropna().unique())
# print(list_gender)
list_person = list(df_EEC['Person'].unique())   
# print(list_person)

In [69]:
# list_f_m_noun_phrase =[]
# list_f_m_noun_phrase.extend(name_male)
# list_f_m_noun_phrase.extend(name_female)
# [list_f_m_noun_phrase.extend([f,m]) for f,m in dict_f_m_noun_phrase.items()]
# print(list_f_m_noun_phrase)
# assert set(list_f_m_noun_phrase)<= set(list_person), "The noun phrases are not subset of overall person list"

In [70]:
print(list_emotion_word)
# list_emotion_word= list_emotion_word.append('')
# print(list_emotion_word)

['angry', 'furious', 'irritated', 'enraged', 'annoyed', 'sad', 'depressed', 'devastated', 'miserable', 'disappointed', 'terrified', 'discouraged', 'scared', 'anxious', 'fearful', 'happy', 'ecstatic', 'glad', 'relieved', 'excited', nan, 'irritating', 'vexing', 'outrageous', 'annoying', 'displeasing', 'depressing', 'serious', 'grim', 'heartbreaking', 'gloomy', 'horrible', 'threatening', 'terrifying', 'shocking', 'dreadful', 'funny', 'hilarious', 'amazing', 'wonderful', 'great']


In [71]:
# Template - F - M Noun Phrases chunks
dict_noun_phrase_sentence_pair = {}
## take a subset where Race field is not populated
df_noun_phrase_subset=  df_EEC[df_EEC['Race'].isna()] ## includes values which do not have Race 
count = 0
# print(len(df_noun_phrase_subset),df_noun_phrase_subset.head())
for template in list_unique_template:
  for emotion_word in list_emotion_word:
    condition_1 = (df_noun_phrase_subset['Template']== template)
    condition_2 = (df_noun_phrase_subset['Emotion word'] == emotion_word)
    condition_3 = (df_noun_phrase_subset['Gender'] == 'female')
    condition_4 = (df_noun_phrase_subset['Gender'] == 'male')
    list_female = df_noun_phrase_subset[(condition_1) & (condition_2) & (condition_3)]['Sentence'].to_list()
    list_male = df_noun_phrase_subset[(condition_1) & (condition_2) & (condition_4)]['Sentence'].to_list()
    if ((len(list_female) >0) & (len(list_male) >0)):
      dict_noun_phrase_sentence_pair[count]=(list_female,list_male)
      # print(emotion_word, "emotion word")
      count = count + 1
    
    if pd.isnull(emotion_word):
      condition_5 = (df_noun_phrase_subset['Emotion word'].isna())
      list_female_2 = df_noun_phrase_subset[(condition_5) & (condition_1) & (condition_3)]['Sentence'].to_list()
      list_male_2 = df_noun_phrase_subset[(condition_5) & (condition_1) & (condition_4)]['Sentence'].to_list()
      if ((len(list_female_2) >0) & (len(list_male_2) >0)):
        dict_noun_phrase_sentence_pair[count]=(list_female_2,list_male_2)
        # print("no emotion word")
        count = count + 1

print(count)
print(len(dict_noun_phrase_sentence_pair),"\ndict_sentence_pair............ \n",dict_noun_phrase_sentence_pair)


144
144 
dict_sentence_pair............ 
 {0: (['She feels angry.', 'This woman feels angry.', 'This girl feels angry.', 'My sister feels angry.', 'My daughter feels angry.', 'My wife feels angry.', 'My girlfriend feels angry.', 'My mother feels angry.', 'My aunt feels angry.', 'My mom feels angry.'], ['He feels angry.', 'This man feels angry.', 'This boy feels angry.', 'My brother feels angry.', 'My son feels angry.', 'My husband feels angry.', 'My boyfriend feels angry.', 'My father feels angry.', 'My uncle feels angry.', 'My dad feels angry.']), 1: (['She feels furious.', 'This woman feels furious.', 'This girl feels furious.', 'My sister feels furious.', 'My daughter feels furious.', 'My wife feels furious.', 'My girlfriend feels furious.', 'My mother feels furious.', 'My aunt feels furious.', 'My mom feels furious.'], ['He feels furious.', 'This man feels furious.', 'This boy feels furious.', 'My brother feels furious.', 'My son feels furious.', 'My husband feels furious.', 'My bo

In [72]:
# # Template - F - M Noun Phrases chunks
# dict_sentence_pair = {}
# count = 0

# for template in list_unique_template:
#   for f, m in dict_f_m_noun_phrase.items():
#     condition_1 = df_EEC['Template']== template
#     condition_2 = df_EEC['Person']== f
#     condition_3 = df_EEC['Person']== m
#     df_temp_f = df_EEC[(condition_1 & condition_2 )] 
#     df_temp_m = df_EEC[(condition_1 & condition_3 )]
#     for emotion_word in list_emotion_word:
      
#       condition_4 = df_EEC['Emotion word'] == emotion_word
      
#       k = df_temp_f[condition_4]['Sentence']
#       v = df_temp_m[condition_4]['Sentence']
#       assert len(k)==len(v), "Problem is in Noun Phase Chunks where emotion_word is not null"
#       if len(k) > 0 and len (v) > 0:
#         dict_sentence_pair[count] = (k.values[0],v.values[0])
#         count = count + 1
      
#       ## Checking for column values where emotion word value blank
#       if pd.isnull(emotion_word):
#         k_null = df_temp_f[df_temp_f['Emotion word'].isna()]['Sentence']
#         v_null = df_temp_m[df_temp_m['Emotion word'].isna()]['Sentence']
#         assert len(k_null)==len(v_null), "Problem is in Noun Phase Chunks where emotion_word is  null"
#         if len(k_null) > 0 and len (v_null) > 0:
#           dict_sentence_pair[count] = (k_null.values[0],v_null.values[0])
#           count = count + 1
      
# print(len(dict_sentence_pair),"\ndict_sentence_pair............ \n",dict_sentence_pair)

In [73]:
# for Named people

dict_list_named_sentence_pairs ={}
df_EEC_subset = df_EEC.dropna(subset = ['Race']) ## removes values which do not have Race 
print(len(df_EEC_subset))

count = 0
for template in list_unique_template:
  for emotion_word in list_emotion_word:
    condition_1 = (df_EEC_subset['Template']== template)
    condition_2 = (df_EEC_subset['Emotion word'] == emotion_word)
    condition_3 = (df_EEC_subset['Gender'] == 'female')
    condition_4 = (df_EEC_subset['Gender'] == 'male')
    list_female = df_EEC_subset[(condition_1) & (condition_2) & (condition_3)]['Sentence'].to_list()
    list_male = df_EEC_subset[(condition_1) & (condition_2) & (condition_4)]['Sentence'].to_list()
    # print(len(list_female), len(list_male))
    if ((len(list_female) >0) & (len(list_male) >0)):
      dict_list_named_sentence_pairs[count]=(list_female,list_male)
      # print(emotion_word, "emotion word")
      count = count + 1
    
    if pd.isnull(emotion_word):
      condition_5 = (df_EEC_subset['Emotion word'].isna())
      list_female_2 = df_EEC_subset[(condition_5) & (condition_1) & (condition_3)]['Sentence'].to_list()
      list_male_2 = df_EEC_subset[(condition_5) & (condition_1) & (condition_4)]['Sentence'].to_list()
      if ((len(list_female_2) >0) & (len(list_male_2) >0)):
        dict_list_named_sentence_pairs[count]=(list_female_2,list_male_2)
        # print("no emotion word")
        count = count + 1
        
    

print (count)
print(len(dict_list_named_sentence_pairs))
print(dict_list_named_sentence_pairs)

5760
144
144
{0: (['Nichelle feels angry.', 'Shereen feels angry.', 'Ebony feels angry.', 'Latisha feels angry.', 'Shaniqua feels angry.', 'Jasmine feels angry.', 'Tanisha feels angry.', 'Tia feels angry.', 'Lakisha feels angry.', 'Latoya feels angry.', 'Amanda feels angry.', 'Courtney feels angry.', 'Heather feels angry.', 'Melanie feels angry.', 'Katie feels angry.', 'Betsy feels angry.', 'Kristin feels angry.', 'Nancy feels angry.', 'Stephanie feels angry.', 'Ellen feels angry.'], ['Alonzo feels angry.', 'Jamel feels angry.', 'Alphonse feels angry.', 'Jerome feels angry.', 'Leroy feels angry.', 'Torrance feels angry.', 'Darnell feels angry.', 'Lamar feels angry.', 'Malik feels angry.', 'Terrence feels angry.', 'Adam feels angry.', 'Harry feels angry.', 'Josh feels angry.', 'Roger feels angry.', 'Alan feels angry.', 'Frank feels angry.', 'Justin feels angry.', 'Ryan feels angry.', 'Andrew feels angry.', 'Jack feels angry.']), 1: (['Nichelle feels furious.', 'Shereen feels furious.', 

In [74]:
print(dict_list_named_sentence_pairs[0][0],"\n",dict_list_named_sentence_pairs[0][1])

['Nichelle feels angry.', 'Shereen feels angry.', 'Ebony feels angry.', 'Latisha feels angry.', 'Shaniqua feels angry.', 'Jasmine feels angry.', 'Tanisha feels angry.', 'Tia feels angry.', 'Lakisha feels angry.', 'Latoya feels angry.', 'Amanda feels angry.', 'Courtney feels angry.', 'Heather feels angry.', 'Melanie feels angry.', 'Katie feels angry.', 'Betsy feels angry.', 'Kristin feels angry.', 'Nancy feels angry.', 'Stephanie feels angry.', 'Ellen feels angry.'] 
 ['Alonzo feels angry.', 'Jamel feels angry.', 'Alphonse feels angry.', 'Jerome feels angry.', 'Leroy feels angry.', 'Torrance feels angry.', 'Darnell feels angry.', 'Lamar feels angry.', 'Malik feels angry.', 'Terrence feels angry.', 'Adam feels angry.', 'Harry feels angry.', 'Josh feels angry.', 'Roger feels angry.', 'Alan feels angry.', 'Frank feels angry.', 'Justin feels angry.', 'Ryan feels angry.', 'Andrew feels angry.', 'Jack feels angry.']


## Two Sample t- test

In [103]:
# Function for t-test processing

def two_sample_test(dict_sentence_pairs ={},text_pipeline = text_pipeline, loaded_model = loaded_model, loaded_model_device = 'cpu')-> dict:
  dict_t_test_result_sentence_pair ={}
  for key, value in dict_sentence_pairs.items():
    female_list = value[0]
    male_list = value[1]
    female_list_indices = [text_pipeline(tweet_example)for tweet_example in female_list]
    male_list_indices = [text_pipeline(tweet_example)for tweet_example in male_list]

    female_list_output = [predict(sentence, loaded_model,text_pipeline,device= loaded_model_device) for sentence in female_list]
    male_list_output = [predict(sentence, loaded_model,text_pipeline,device= loaded_model_device) for sentence in male_list]

    t_test_result = stats.ttest_rel(female_list_output, male_list_output)
    dict_t_test_result_sentence_pair[key] = (t_test_result.statistic, t_test_result.pvalue,mean(female_list_output)-mean(male_list_output))
  # print(dict_t_test_result_sentence_pair)
  return dict_t_test_result_sentence_pair



In [104]:
list_sentence_pairs = ['named','noun_phrase']
dict_t_test ={}
for model_type, loaded_model in dict_loaded_model.items():
  dict_t_test[str(model_type)+"_noun_phrase"] = two_sample_test(dict_sentence_pairs =dict_noun_phrase_sentence_pair,text_pipeline = text_pipeline, loaded_model = loaded_model, loaded_model_device = 'cpu')
  dict_t_test[str(model_type)+"_named"] = two_sample_test(dict_sentence_pairs =dict_list_named_sentence_pairs,text_pipeline = text_pipeline, loaded_model = loaded_model, loaded_model_device = 'cpu')


In [ ]:
# dict_t_test.items()

In [ ]:
# dict_t_test_noun_phrase_sentence_pair = two_sample_test(dict_sentence_pairs =dict_noun_phrase_sentence_pair,text_pipeline = text_pipeline, loaded_model = loaded_model, loaded_model_device = 'cpu')
# dict_t_test_named_sentence_pairs = two_sample_test(dict_sentence_pairs =dict_list_named_sentence_pairs,text_pipeline = text_pipeline, loaded_model = loaded_model, loaded_model_device = 'cpu')

In [77]:
# dict_result_named_sentence_pair ={}

# for key, value in dict_list_named_sentence_pairs.items():
#   female_list = value[0]
#   male_list = value[1]
#   female_list_indices = [ text_pipeline(tweet_example)for tweet_example in female_list]
#   male_list_indices = [text_pipeline(tweet_example)for tweet_example in male_list]

#   female_list_output = [predict(sentence, loaded_model,text_pipeline,device= loaded_model_device) for sentence in female_list]
#   male_list_output = [predict(sentence, loaded_model,text_pipeline,device= loaded_model_device) for sentence in male_list]
#   # for sentence in female_list:
#   #   female_list_output.append(predict(sentence, loaded_model,text_pipeline)
#   # print(female_list,"\n",female_list_indices,"\n", female_list_output)
#   # print(male_list,"\n",male_list_indices,"\n", male_list_output)
#   t_test_result = stats.ttest_rel(female_list_output, male_list_output)
#   dict_result_named_sentence_pair[key] = (t_test_result.statistic, t_test_result.pvalue,mean(female_list_output)-mean(male_list_output))
#   # print(type(stats.ttest_rel(female_list_output, male_list_output)))

#   # break

# print((dict_result_named_sentence_pair))

In [78]:
# #without named people
# dict_result_sentence_pair ={}
# # for key, value in dict_sentence_pair:
# #   if len(value[0])
# print(len(dict_sentence_pair))

# for key, value in dict_sentence_pair.items():
#   female_list = [value[0]]
#   male_list = [value[1]]
#   # if len(female_list)!=len(male_list):
#   #   print("key:", key)
#   #   print(female_list,"\n",male_list)
#   #   print(len(female_list),"-",len(male_list))
#   #   print(text_pipeline(female_list[0]),"\n",text_pipeline(male_list[0]))
#   #   break

#   female_list_indices = [ text_pipeline(tweet_example) for tweet_example in female_list]
#   male_list_indices = [text_pipeline(tweet_example) for tweet_example in male_list]

#   female_list_output = [predict(sentence, loaded_model,text_pipeline,device= loaded_model_device) for sentence in female_list]
#   male_list_output = [predict(sentence, loaded_model,text_pipeline,device= loaded_model_device) for sentence in male_list]
#   # for sentence in female_list:
#   #   female_list_output.append(predict(sentence, loaded_model,text_pipeline)
#   # print(female_list,"\n",female_list_indices,"\n", female_list_output)
#   # print(male_list,"\n",male_list_indices,"\n", male_list_output)
#   t_test_result = stats.ttest_rel(female_list_output, male_list_output)
#   dict_result_sentence_pair[key] = (t_test_result.statistic, t_test_result.pvalue,mean(female_list_output)-mean(male_list_output))
#   # print(type(stats.ttest_rel(female_list_output, male_list_output)))

#   # break

# print(dict_result_sentence_pair)

# Analysis of results (based on semval paper)

In [83]:
# dict_t_test_noun_phrase_sentence_pair
# dict_t_test_named_sentence_pairs

In [80]:
# len(dict_t_test_noun_phrase_sentence_pair),len(dict_t_test_named_sentence_pairs)

(144, 144)

In [106]:
def analysis_t_test(dict_t_test_sentence_pairs, threshold = 0.05):
  list_output =[]
  for key, test_output in dict_t_test_sentence_pairs.items():
    significant=True
    t_statistic = test_output[0]
    p_value = test_output[1]
    f_m_diff = test_output[2]
    if (float(p_value) > float(threshold) or float(p_value) == float(threshold)):
      significant=False
      category = 'f_equals_m'
    else:
      significant=True
      
      if f_m_diff > 0:
        category='f_high_m_low'
      else:
        category = 'f_low_m_high' 
    list_output.append([key,t_statistic,p_value,significant,f_m_diff,category])
    
  df_columns = ['key','t_statistic','p_value', 'significant','delta','category']
  df_output = pd.DataFrame(list_output, columns = df_columns)

  list_category = list(df_output['category'].unique())
  list_statistics =[]
  for category in list_category:
    df_temp = df_output[df_output['category']==category]
    average = df_temp['delta'].mean()
    # print(category,len(df_temp), average)
    list_statistics.append([category,len(df_temp), average])
  df_statistics = pd.DataFrame(list_statistics, columns = ['category', 'num_pairs','average_difference'])
  return df_statistics


# print(analysis_t_test(dict_t_test_noun_phrase_sentence_pair))
# print(analysis_t_test(dict_t_test_named_sentence_pairs))


In [107]:
dict_statistics={}
for model_type_sentence_pair_name, t_test_dict in dict_t_test.items():
  df_statistics = analysis_t_test(t_test_dict, threshold = 0.05)
  dict_statistics[model_type_sentence_pair_name] = df_statistics


In [110]:
for model_type_sentence_pair_name, df_statistics in dict_statistics.items():
  print(model_type_sentence_pair_name,"\n",df_statistics)
  print(50*"=")

non_dann_noun_phrase 
        category  num_pairs  average_difference
0    f_equals_m        126            0.005233
1  f_high_m_low         18            0.026457
non_dann_named 
        category  num_pairs  average_difference
0    f_equals_m        124            0.008112
1  f_high_m_low         20            0.023590
dann_noun_phrase 
        category  num_pairs  average_difference
0    f_equals_m        124            0.006045
1  f_high_m_low         20            0.021322
dann_named 
        category  num_pairs  average_difference
0    f_equals_m        121            0.008845
1  f_high_m_low         23            0.020125
